## Intraday Example

In [1]:
import os

os.chdir("..")

### Generating random minute level price data

In [2]:
import random
from datetime import datetime, timedelta
import pandas as pd
import cufflinks as cf


random.seed(0)
price = 100
current_time = datetime(2020, 11, 1, 10)
end_time = datetime(2020, 11, 1, 16)

times, prices = [], []
while current_time <= end_time:
    times.append(current_time)
    prices.append(price)
    current_time += timedelta(minutes=1)
    price += random.random() - 0.5

df = pd.DataFrame(index=times, data=prices, columns=["price"])
df["SMA"] = df.price.rolling(30).mean()
df

,price,SMA
2020-11-01 10:00:00,100.000000,NaN
2020-11-01 10:01:00,100.344422,NaN
2020-11-01 10:02:00,100.602376,NaN
2020-11-01 10:03:00,100.522948,NaN
2020-11-01 10:04:00,100.281865,NaN
...,...,...
2020-11-01 15:56:00,103.313055,103.418283
2020-11-01 15:57:00,103.153275,103.392886
2020-11-01 15:58:00,103.615842,103.369473
2020-11-01 15:59:00,104.014850,103.360974


In [3]:
df.iplot(title="Prices")

### Setting up our backtest
Initialise our assets and the portfolio.

In [4]:
from pytrade import Trade, Strategy, Backtest, History
from pytrade.assets import reset, Cash, Stock, Portfolio
from pytrade.compliance import Compliance, UnitLimit

reset()
stock = Stock("XXX US", currency_code="USD")
usd = Cash("USD")
portfolio = Portfolio("USD")
portfolio.transfer(usd, int(1e6))

portfolio.value

1000000.0

Impose a holding limit of 1000 shares for stock 'XXX US'.

In [5]:
portfolio.compliance = Compliance().add_rule(
    UnitLimit(stock, 1000)
)

Define a trading strategy.

In [6]:
class IntradayStrategy(Strategy):
    """ Keep buying +100 shares where price > sma.
        Start selling your existing position, 100 shares at a time, when price < sma.
    """
    def generate_trades(self):
        sma = backtest.get_indicator("SMA")
        if sma is None:  # don't trade if we don't yet have sma data
            return

        price = stock.price
        position = portfolio.get_holding_units("XXX US")
        if price > sma:  # establish a position
            return Trade(portfolio, stock, +100)

        if price < sma and position > 0:  # then start selling your position
            return Trade(portfolio, stock, -100)


Create a backtest object and load our random data as events.

In [7]:
backtest = Backtest(IntradayStrategy())

In [8]:
from pytrade.events import load_frame_events, AssetPriceEvent, IndicatorEvent

load_frame_events(
    stock,                       # load events for stock 'XXX US'
    df, "price",                 # from dataframe 'df', column 'price'
    backtest=backtest,           # for this backtest
    event_class=AssetPriceEvent, # and of Event type AssetPriceEvent
)

load_frame_events(
    "SMA",                       # load indicator events into the backtest under the code 'SMA'
    df, "SMA",                   # from dataframe 'df', column 'SMA'
    backtest=backtest,           # for this backtest
    event_class=IndicatorEvent,  # and of Event type IndicatorEvent
)

361

### Run the backtest and view results

In [9]:
history = History(
    portfolios=portfolio,
    backtest=backtest,
)

backtest.run()
df = history.get()

df

,Portfolio,Portfolio_USD,Portfolio_XXX US,SMA,USD,XXX US
2020-11-01 10:00:00,1.000000e+06,1000000.000000,0.0,NaN,1.0,100.000000
2020-11-01 10:01:00,1.000000e+06,1000000.000000,0.0,NaN,1.0,100.344422
2020-11-01 10:02:00,1.000000e+06,1000000.000000,0.0,NaN,1.0,100.602376
2020-11-01 10:03:00,1.000000e+06,1000000.000000,0.0,NaN,1.0,100.522948
2020-11-01 10:04:00,1.000000e+06,1000000.000000,0.0,NaN,1.0,100.281865
...,...,...,...,...,...,...
2020-11-01 15:56:00,1.000896e+06,938908.070131,600.0,103.418283,1.0,103.313055
2020-11-01 15:57:00,1.000800e+06,949223.397659,500.0,103.392886,1.0,103.153275
2020-11-01 15:58:00,1.001031e+06,938861.813485,600.0,103.369473,1.0,103.615842
2020-11-01 15:59:00,1.001271e+06,928460.328507,700.0,103.360974,1.0,104.014850


In [14]:
import cufflinks as cf

columns = ["Portfolio_XXX US", "XXX US", "SMA"]

df[columns].iplot(
    secondary_y=["XXX US", "SMA"],
    title="Intraday Holdings",
)

In [15]:
columns = ["Portfolio", "XXX US", "SMA"]

df[columns].iplot(
    secondary_y=["XXX US", "SMA"],
    title="Portfolio Value (USD)",
)

***